# Robust Regression

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn import linear_model

## Outliers

In [ ]:
# Let's use the same data from earlier this morning and add some error
n = 100
x = np.arange(1,n).reshape(-1,1)
y = np.array([(i**2)+(10*i)*(np.sin(i)+1) for i in x])
plt.scatter(x,y)

In [ ]:
# Add some problem data points
noutlier = 20

By making the number of outliers a variable it is easy to adjust and rerun to see how the number of outliers changes the ability to fit a model

In [ ]:
x = 
y = 

In [ ]:
plt.scatter(x,y)

In [ ]:
def make_plots(title=None):
    """
    Plot residual and model fit plots. The assumption of outside
    function variable names that this function is based on is bad practice.
    """

## Regression Model

Be sure to apply the transformation that we found earlier and separate out test and training sets

### General regression model

In [ ]:
model = 
model.fit()
print("R^2: ",model.score(x_test, y_test))
print("Slope: ", model.coef_)
print("Intercept: ", model.intercept_)
pred_test = 
res_test = 
pred = 
res = 

In [ ]:
make_plots('OLS Regression')

## RANSAC

In [ ]:
ransac = linear_model.RANSACRegressor()
ransac.fit()
print("R^2: ",ransac.score(x_test, y_test))
pred_test = 
res_test = 
pred = 
res = 

In [ ]:
make_plots('RANSAC Regression')

## Theil-Sen

In [ ]:
theil_sen = linear_model.TheilSenRegressor(random_state=3)
theil_sen.fit(x_train, y_train.ravel())
print("R^2: ",theil_sen.score(x_test, y_test))
pred_test = 
res_test = 
pred = 
res = 

In [ ]:
make_plots('Theil-Sen Regression')

## Huber 

In [ ]:
huber = linear_model.HuberRegressor()
huber.fit(x_train, y_train.ravel())
print("R^2: ",huber.score(x_test, y_test))
pred_test = 
res_test = 
pred = 
res = 

In [ ]:
make_plots('Huber Regression')

### Comparison

In [ ]:
print("R^2 OLS: ",model.score(x_test, y_test))
print("R^2 RANSAC: ",ransac.score(x_test, y_test))
print("R^2 Theil-Sen: ",theil_sen.score(x_test, y_test))
print("R^2 Huber: ",huber.score(x_test, y_test))

In [ ]:
plt.scatter(x,y,label='Transformed Data')
xseq = np.linspace(0,100,num=100).reshape(-1, 1)
plt.plot(xseq,model.predict(xseq),label='OLS')
plt.plot(xseq,ransac.predict(xseq),label='RANSAC')
plt.plot(xseq,theil_sen.predict(xseq),label='Theil-Sen')
plt.plot(xseq,huber.predict(xseq),label='Huber')
plt.legend()

In [ ]:
plt.scatter(x,y**2,label='Data')
xseq = np.linspace(0,100,num=100).reshape(-1, 1)
plt.plot(xseq,model.predict(xseq)**2,label='OLS')
plt.plot(xseq,ransac.predict(xseq)**2,label='RANSAC')
plt.plot(xseq,theil_sen.predict(xseq)**2,label='Theil-Sen')
plt.plot(xseq,huber.predict(xseq)**2,label='Huber')
plt.legend()